In [2]:
import pprint
from pathlib import Path
from pprint import pprint

import torch
import yaml
from easydict import EasyDict

from data_loader_manager import dataloaders
from models import vgg_model
from trainers.metrics_processor import MetricsProcessor

with open("../configs/igs_config.yaml", "r") as file:
    config = yaml.safe_load(file)


pprint(config)

config = EasyDict(config)
config.aug = False
config.adversarial = False

config.sharpness_batch_size = 16
config.sharpness_dataset_size = 128*10

device = "cuda" if torch.cuda.is_available() else "cpu"

print(device)

model = vgg_model.VGG(
    vgg_name="VGG19",
    dropout=0.0,
    vgg_config=Path(config.config_dir) / config.vgg_config,
    num_classes=10
)
model = model.to(device)

seed = 42

{'batch_size': 256,
 'cifar_dir': '../data/cifar',
 'config_dir': '../configs',
 'learning_rate': 0.05,
 'metrics': ['IGS'],
 'models_dir': '../models',
 'momentum': 0.9,
 'num_epochs': 75,
 'seed': [43, 91, 17],
 'sharpness_batch_size': 36,
 'sharpness_dataset_size': 750,
 'vgg_config': 'vgg_config.json',
 'wandb': {'entity': 'r252_project',
           'experiment_name': '',
           'project': 'VGG19_CIFAR100_FINAL'}}


In [3]:
model_dir = Path("../models/CIFAR10/")

In [6]:
import glob

import numpy as np

for model_path in glob.glob(f"{str(model_dir)}/*/*/best_baseline*.pth"):
  print(model_path)
  model.load_state_dict(torch.load(model_path, map_location=device))
  data_loader_manager = dataloaders.DataLoaderManager(
      config=config,
      dataset_name="CIFAR10",
      seed=seed,
  )

  train_dataloader, dev_dataloader, test_dataloader, sharpness_dataloader = data_loader_manager.get_dataloaders()

  model.eval()
  mp = MetricsProcessor(
      config=config,
      model=model,
      train_dataloader=sharpness_dataloader,
      test_dataloader=sharpness_dataloader,
      model_name="",
      device=device,
      seed=seed,
      num_classes=data_loader_manager.num_classes
  )

  igs = np.log(mp.IGS(output_all=True))
  print(igs)
  print(igs.mean(), igs.std())

  with (model_dir / "igs_output").open("a") as logfile:
      logfile.write(f"{model_path},{igs.mean()},{igs.std()}\n")

../models/CIFAR10/baseline/91/best_baseline.pth
Loading normal train data
Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/80 [00:00<?, ?it/s]/rds/user/is473/hpc-work/R252_Group_Project/src/trainers/igs/igs.py:1329: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:83.)
  y_F = torch.cuda.LongTensor([i]*output.shape[0])
  0%|          | 0/80 [00:05<?, ?it/s]
/rds/user/is473/hpc-work/R252_Group_Project/src/trainers/metrics_processor.py:189: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  assert i is not -1
/rds/user/is473/hpc-work/R252_Group_Project/src/trainers/metrics_processor.py:189: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  assert i is not -1


RuntimeError: No CUDA GPUs are available